In [16]:
import pandas as pd
from neo4j import GraphDatabase

path = "data_test.csv"
df = pd.read_csv(path, sep=";")

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "neo4jneo4j"))

# Создаем узлы типа "Событие"
with driver.session() as session:
    for row in df.itertuples(index=False):
        event_id = row[0]
        session.run("MERGE (e:Event {id_sobytiya: $event_id})", event_id=event_id)

# Создаем узлы типа "Участник" и связи "Участник_события"
with driver.session() as session:
    for row in df.itertuples(index=False):
        event_id = row[0]
        participant_1 = row[1]
        participant_2 = row[2]
        session.run("""
            MERGE (p1:Participant {fio: $participant_1})
            MERGE (p2:Participant {fio: $participant_2})
            MERGE (p1)-[:PARTICIPATED_IN {event_id: $event_id}]->(e:Event {id_sobytiya: $event_id})
            MERGE (p2)-[:PARTICIPATED_IN {event_id: $event_id}]->(e)
        """, participant_1=participant_1, participant_2=participant_2, event_id=event_id)